<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/clinical_data_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDC clinical data exploration

The goal of this notebook is to introduce the users of NCI Imaging Data Commons (IDC) to the organization of clinical data BigQuery tables that accompany some of the IDC imaging data. For a quick visual summary of the clinical data available in IDC, please check out [this DataStudio dashboard](https://datastudio.google.com/u/0/reporting/04cf5976-4ea0-4fee-a749-8bfd162f2e87/page/p_s7mk6eybqc).

[NCI Imaging Data Commons (IDC)](https://imaging.datacommons.cancer.gov) is a cloud-based repository of publicly available cancer imaging data co-located with the analysis and exploration tools and resources. IDC is a node within the broader [NCI Cancer Research Data Commons (CRDC)](https://datacommons.cancer.gov/) infrastructure that provides secure access to a large, comprehensive, and expanding collection of cancer research data.

If you are not familiar with IDC, we recommend you first take a look at the [Getting started](https://github.com/ImagingDataCommons/IDC-Examples/blob/master/notebooks/getting_started.ipynb) notebook that is intended to serve as the introduction into working with IDC programmatically.

If you have any questions about this tutorial, please post your questions on the [IDC user forum](https://discourse.canceridc.dev/) (preferred) or email IDC support at support@canceridc.dev!

Authored by Andrey Fedorov and George White

Prepared: July 2022

Updated: Sept 2022

# Prerequisites

In order to be able to run the cells in this notebook, you must complete the prerequisites to set up your Google Cloud Platform account, as discussed here: https://learn.canceridc.dev/introduction/getting-started-with-gcp.

Once you completed the prerequisites, insert your Google Cloud Platform project ID in the cell below in place of `REPLACE_ME_WITH_YOUR_PROJECT_ID`.

In [ ]:
# initialize this variable with your Google Cloud Project ID!
my_ProjectID = "REPLACE_ME_WITH_YOUR_PROJECT_ID"

import os
os.environ["GCP_PROJECT_ID"] = my_ProjectID

!gcloud config set project $GCP_PROJECT_ID

Updated property [core/project].


In the following cell you will be asked to authorize Google Collaboratory to act on your behalf. In order to proceed with the following cells you must allow this.

In [ ]:
# you will need to authenticate with your Google ID to do anything meaningful with IDC
from google.colab import auth
auth.authenticate_user()

# Clinical data - background

By clinical data we refer to the broad spectrum of image-related data that may accompany images. Such data may include demographics of the patients, observations related to their clinical history (therapies, diagnoses, findings), lab tests, surgeries.

Clinical data is often critical in understanding imaging data, and is essential for the development and validation of imaging biomarkers. However, such data is most often stored in spreadsheets that follow conventions specific to the site that collected the data, may not be accompanied by the dictionary defining the terms used in describing clinical data, and is rarely harmonized. As an example, you can consider examing the clinical data that accompanies the [ACRIN 6698 collection](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=50135447) curated by The Cancer Imaging Archive (TCIA). File named "Full Ancillary Patient Information file.xlsx" linked from the collection page contains two sheets, one of which is the dictionary, and the othe one contains per-patient data with the columns defined by the dictionary.

Not only the terms used in the clinical data accompanying individual collection are not harmonized, but the format of the spreadsheets is also collection-specific. In order to search and navigate clinical data, one has to parse those collection specific tables, and there is no interface to support searching across collections.

With the release v11 of IDC, we make the attempt to lower the barriers for accessing clinical data accompanying IDC imaging collections. We parse collection-specific tables, and organize the underlying data into BigQuery tables that can be accessed using standard SQL queries. You can also see the summary of clinical data available for IDC collections in [this dashboard](https://datastudio.google.com/u/0/reporting/04cf5976-4ea0-4fee-a749-8bfd162f2e87/page/p_s7mk6eybqc). 

At the completion of this tutorial you will learn how IDC clinical data is organized, and how to write queries to interrogate this data.

# Organization of clinical data in IDC

Clinical data accompanying imaging collections is organized into collection-specific tables in the `idc_current_clinical` BigQuery dataset (in order to locate this dataset, you will need to navigate to the [BigQuery console](https://console.cloud.google.com/bigquery), and locate `idc_current_clinical` under the `bigquery-public-data` project, which you should have added in the prerequisites mentioned earlier. 

When a certain imaging collection is accompanied by a single clinical sheet, the BigQuery table will be named `<collection_id>_clinical`. For the collection that have multiple clinical sheets (saved as either separate files, or separate Excel sheets), the tables will be named `<collection_id>_<sheet_id>`. 

## Inventory of clinical data tables: `table_metadata`

In [ ]:
# python API is the most flexible way to query IDC BigQuery metadata tables
from google.cloud import bigquery
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
SELECT *
FROM bigquery-public-data.idc_v11_clinical.table_metadata
ORDER BY collection_id
"""

selection_result = bq_client.query(selection_query)
table_metadata_df = selection_result.result().to_dataframe()

table_metadata_df

,collection_id,table_name,table_description,idc_version_table_added,table_added_datetime,post_process_src,post_process_src_added_md5,idc_version_table_prior,post_process_src_prior_md5,idc_version_table_updated,table_updated_datetime,post_process_src_updated_md5,number_batches,source_info
0,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,clinical data,idc_v11,2022-09-02 04:08:47.706090+00:00,././clin_idc_v11//acrin_6698_clinical.json,8664e8bb2dd37fc196e0bbcce608b9c0,idc_v11,8664e8bb2dd37fc196e0bbcce608b9c0,idc_v11,2022-09-02 04:08:47.706115+00:00,8664e8bb2dd37fc196e0bbcce608b9c0,1,[{'srcs': ['Full Collection Ancillary Patient ...
1,acrin_flt_breast,bigquery-public-data.idc_v11_clinical.acrin_fl...,*HQ*CONFIRMATION,idc_v11,2022-09-02 04:08:47.707085+00:00,././clin_idc_v11//acrin_flt_breast_A0.csv,2a6572702bc6a9fcf8fd4a6b8543ae86,idc_v11,2a6572702bc6a9fcf8fd4a6b8543ae86,idc_v11,2022-09-02 04:08:47.707097+00:00,2a6572702bc6a9fcf8fd4a6b8543ae86,1,[{'srcs': ['ACRIN-FLT-Breast 6688 Clinical Dat...
2,acrin_flt_breast,bigquery-public-data.idc_v11_clinical.acrin_fl...,*HQ*COMMENT,idc_v11,2022-09-02 04:08:47.707985+00:00,././clin_idc_v11//acrin_flt_breast_A1.csv,a0bb2d626a62d4e81fcf5edc3e82b182,idc_v11,a0bb2d626a62d4e81fcf5edc3e82b182,idc_v11,2022-09-02 04:08:47.707997+00:00,a0bb2d626a62d4e81fcf5edc3e82b182,1,[{'srcs': ['ACRIN-FLT-Breast 6688 Clinical Dat...
3,acrin_flt_breast,bigquery-public-data.idc_v11_clinical.acrin_fl...,HQ Study Chair Review,idc_v11,2022-09-02 04:08:47.708637+00:00,././clin_idc_v11//acrin_flt_breast_A3.csv,475362d17db263fcf81275d072c5bca9,idc_v11,475362d17db263fcf81275d072c5bca9,idc_v11,2022-09-02 04:08:47.708649+00:00,475362d17db263fcf81275d072c5bca9,1,[{'srcs': ['ACRIN-FLT-Breast 6688 Clinical Dat...
4,acrin_flt_breast,bigquery-public-data.idc_v11_clinical.acrin_fl...,ELIGIBILITY CHECKLIST,idc_v11,2022-09-02 04:08:47.709357+00:00,././clin_idc_v11//acrin_flt_breast_AA.csv,0dccc58f9531a674fc351dc9447e8390,idc_v11,0dccc58f9531a674fc351dc9447e8390,idc_v11,2022-09-02 04:08:47.709369+00:00,0dccc58f9531a674fc351dc9447e8390,1,[{'srcs': ['ACRIN-FLT-Breast 6688 Clinical Dat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,tcga_thca,bigquery-public-data.idc_v11_clinical.tcga_thc...,clinical_data,idc_v11,2022-09-02 04:18:00.448221+00:00,None,None,idc_v11,None,idc_v11,2022-09-02 04:18:00.448238+00:00,None,1,[{'srcs': ['bigquery-public-data.idc_v11.tcga_...
185,tcga_thym,bigquery-public-data.idc_v11_clinical.tcga_thy...,clinical_data,idc_v11,2022-09-02 04:18:04.083044+00:00,None,None,idc_v11,None,idc_v11,2022-09-02 04:18:04.083061+00:00,None,1,[{'srcs': ['bigquery-public-data.idc_v11.tcga_...
186,tcga_ucec,bigquery-public-data.idc_v11_clinical.tcga_uce...,clinical_data,idc_v11,2022-09-02 04:18:07.941624+00:00,None,None,idc_v11,None,idc_v11,2022-09-02 04:18:07.941643+00:00,None,1,[{'srcs': ['bigquery-public-data.idc_v11.tcga_...
187,tcga_ucs,bigquery-public-data.idc_v11_clinical.tcga_ucs...,clinical_data,idc_v11,2022-09-02 04:18:11.782022+00:00,None,None,idc_v11,None,idc_v11,2022-09-02 04:18:11.782040+00:00,None,1,[{'srcs': ['bigquery-public-data.idc_v11.tcga_...


The per-row attributes stored in this table aim to describe the content of the specific clinical table, its versioning and provenance (what specific file/sheet was used to populate the content of the table).

Column `table_name` contains the fully resolved pointer to the BigQuery table with the parsed clinical data.

## Inventory of clinical data "dictionary terms": `column_metadata`

The purpose of the clinical data "dictionary terms" is to to facilitate exploration of and access to the clinical data. Although the content of the clinical data sheets is for the most part not harmonized, they follow the same general structure:

1. files/sheets that contain per-patient rows of clinical data attributes, defined by column headers
2. (in some cases) file/sheet defining the meaning of the individual columns and (in some cases, and when applicable) the dictionary defining the enumerated values for the specific

The IDC inventory of the clinical data "dictionary terms" contains the complete list of column names, across all clinical sheets, and per-column attributes to facilitate exploration. Let's look at the content to better understand what this means.

In [ ]:
selection_query = """
SELECT *
FROM bigquery-public-data.idc_v11_clinical.column_metadata
ORDER BY collection_id
"""

selection_result = bq_client.query(selection_query)
column_metadata_df = selection_result.result().to_dataframe()

In [ ]:
column_metadata_df.columns

Index(['collection_id', 'case_col', 'table_name', 'column', 'column_label',
       'data_type', 'original_column_headers', 'values', 'files',
       'sheet_names', 'batch', 'column_numbers'],
      dtype='object')

The only column that is initialized consistently across those collection-specific tables is `dicom_patient_id`, which corresponds to the `PatientID` column in the `dicom_all` table. `dicom_patient_id` is the only column that is curated by IDC, since the original clinical data tables do not follow the consistent convention while referring to the patient identification.

Note that some imaging collections do not have any clinical data at all, while others may have multiple tables containing various types of clinical data.

`column_metadata` should be used as a "key" for identifying information of interest in clinical metadata. It contains information about the columns across all of the collections for which we have clinical data.

The most important columns in this table are the following:
* `column`: name of the column in a collection-specific clinical metadata table; note that for some of the collections this name may not contain any human readable information, and may instead be a coded value
* `column_label`: for some collections it may contain a description for a column name that otherwise may not be human readable. For other collections it is the original name of the column in the source Excel file
* `collection_id` and `table_name` identify the collection the specific column corresponds to, and the name of the clinical data table where this column is present.



Here's an example of the columns that correspond to the clinical metadata in one of the ACRIN collections, where `column` is not particularly helpful, but `column_label` provides human readable information to allow interpretation of the column.

In [ ]:
column_metadata_df[column_metadata_df["collection_id"] == "acrin_nsclc_fdg_pet"][:5]

For some columns, the values come from a defined set. In the example above, we can, for example, examine the values assigned to encode patient race.

In [ ]:
race_values = column_metadata_df[(column_metadata_df["collection_id"] == "acrin_nsclc_fdg_pet") & (column_metadata_df["column_label"] == "RACE")]["values"]

race_values.tolist()

[[{'option_code': '1',
   'option_description': 'American Indian or Alaska Native'},
  {'option_code': '2', 'option_description': 'Asian'},
  {'option_code': '3', 'option_description': 'Black or African American'},
  {'option_code': '4',
   'option_description': 'Native Hawaiian or other Pacific Islander'},
  {'option_code': '5', 'option_description': 'White'},
  {'option_code': '6', 'option_description': 'More than one race'},
  {'option_code': '9', 'option_description': 'Unknown'}]]

On the other hand, if we look at the metadata available for the `c4kc_kits` collection, `column_label` and `column` are identical. 


In [ ]:
column_metadata_df[column_metadata_df["collection_id"] == "c4kc_kits"][:3]

**As a general rule of thumb**:
* when selecting specific columns from clinical tables, use `column` values
* when searching for concepts of interest in `column_metadata`, use `column_label`

# Exploring IDC clinical data

In the following sections of the notebook we go over some use cases to demonstrate various options for navigating IDC clinical data. 

As always, if you have a use case that is not addressed here, if you have suggestions or are confused - please start a discussion thread on the [IDC User forum](https://discourse.canceridc.dev/)!

## Collection-focused exploration

If you used clinical data stored in TCIA, perhaps you started with a specific collection of your interest, downloaded clinical data files for that collection, parsed them into your code. Let's go over those steps the IDC way!

First, let's see which of the collections in IDC have clinical data in BigQuery. To do that we will reuse the pandas dataframe with the results of the query we executed earlier.


In [ ]:
column_metadata_df["collection_id"].unique().tolist()

['acrin_6698',
 'acrin_flt_breast',
 'acrin_fmiso_brain',
 'acrin_hnscc_fdg_pet_ct',
 'acrin_nsclc_fdg_pet',
 'b_mode_and_ceus_liver',
 'breast_diagnosis',
 'breast_mri_nact_pilot',
 'c4kc_kits',
 'cmmd',
 'covid_19_ar',
 'covid_19_ny_sbu',
 'cptac_brca',
 'cptac_ccrcc',
 'cptac_coad',
 'cptac_gbm',
 'cptac_hnscc',
 'cptac_lscc',
 'cptac_luad',
 'cptac_ov',
 'cptac_pda',
 'cptac_ucec',
 'hcc_tace_seg',
 'head_neck_pet_ct',
 'head_neck_radiomics_hn1',
 'hnscc',
 'hnscc_3dct_rt',
 'ispy1',
 'ispy2',
 'lidc_idri',
 'lung_fused_ct_pathology',
 'lung_pet_ct_dx',
 'midrc_ricord_1a',
 'midrc_ricord_1b',
 'midrc_ricord_1c',
 'nsclc_radiogenomics',
 'nsclc_radiomics',
 'nsclc_radiomics_genomics',
 'nsclc_radiomics_interobserver1',
 'opc_radiomics',
 'prostate_diagnosis',
 'qin_breast',
 'qin_headneck',
 'rembrandt',
 'soft_tissue_sarcoma',
 'spie_aapm_lung_ct_challenge',
 'tcga_acc',
 'tcga_blca',
 'tcga_brca',
 'tcga_cesc',
 'tcga_chol',
 'tcga_coad',
 'tcga_dlbc',
 'tcga_esca',
 'tcga_gbm',
 

If you are interested in what clinical data is available for the specific collection, you can select only the rows corresponding to that collection in the `column_metadata` table. Here we select a subset of columns to improve readability of the dataframe.

Note that for some collections, clinical data sheets are accompanied by dictionaries, which formalize the values encountered. Examples of such collections are [ISPY1](https://wiki.cancerimagingarchive.net/display/Public/ISPY1) or ACRIN trials.

For many other collections there are no such dictionaries available. In those situations, the values you will see in the `values` columns have been derived by examining the distinct values encountered in the clinical data sheets.

In the following we look at the clinical data columns ("dictionary terms") for the [ACRIN 6698 collection](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=50135447) mentioned earlier.

In [ ]:
acrin6698_clinical_df = column_metadata_df[column_metadata_df["collection_id"] == "acrin_6698"] 

acrin6698_clinical_df

,collection_id,case_col,table_name,column,column_label,data_type,original_column_headers,values,files,sheet_names,batch,column_numbers
0,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,t0,T0 (baseline) MRI study included in collection,int64,[['T0']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[2]
1,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,t1,T1 (early-Tx) MRI study included in collection,int64,[['T1']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[3]
2,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,t2,T2 (inter-regimen) MRI study included in colle...,int64,[['T2']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[4]
3,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,t3,T3 (pre-surgery) MRI study included in collection,int64,[['T3']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[5]
4,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,bmmr2_train,Patient included in the BMMR2 challenge traini...,int64,[['BMMR2_TRAIN']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[6]
5,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,bmmr2_test,Patient included in the BMMR2 challenge test c...,int64,[['BMMR2_TEST']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[7]
6,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t0,T0 MRI study included in the ACRIN-6698 primar...,int64,[['PRIMARY_AIM_T0']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[8]
7,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t1,T1 MRI study included in the ACRIN-6698 primar...,int64,[['PRIMARY_AIM_T1']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[9]
8,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t2,T2 MRI study included in the ACRIN-6698 primar...,int64,[['PRIMARY_AIM_T2']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[10]
9,acrin_6698,False,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t3,T3 MRI study included in the ACRIN-6698 primar...,int64,[['PRIMARY_AIM_T3']],"[{'option_code': '0', 'option_description': ' ...",[Full Collection Ancillary Patient Information...,[ACRIN_6698_Patient_Cohorts_Clin],[0],[11]


Here's how you can select just the specific columns in the table - this way it is easier to examine the data.

In [ ]:
acrin6698_clinical_df[["collection_id", "table_name", "column", "column_label", "data_type", "values"]]

,collection_id,table_name,column,column_label,data_type,values
0,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,t0,T0 (baseline) MRI study included in collection,int64,"[{'option_code': '0', 'option_description': ' ..."
1,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,t1,T1 (early-Tx) MRI study included in collection,int64,"[{'option_code': '0', 'option_description': ' ..."
2,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,t2,T2 (inter-regimen) MRI study included in colle...,int64,"[{'option_code': '0', 'option_description': ' ..."
3,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,t3,T3 (pre-surgery) MRI study included in collection,int64,"[{'option_code': '0', 'option_description': ' ..."
4,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,bmmr2_train,Patient included in the BMMR2 challenge traini...,int64,"[{'option_code': '0', 'option_description': ' ..."
5,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,bmmr2_test,Patient included in the BMMR2 challenge test c...,int64,"[{'option_code': '0', 'option_description': ' ..."
6,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t0,T0 MRI study included in the ACRIN-6698 primar...,int64,"[{'option_code': '0', 'option_description': ' ..."
7,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t1,T1 MRI study included in the ACRIN-6698 primar...,int64,"[{'option_code': '0', 'option_description': ' ..."
8,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t2,T2 MRI study included in the ACRIN-6698 primar...,int64,"[{'option_code': '0', 'option_description': ' ..."
9,acrin_6698,bigquery-public-data.idc_v11_clinical.acrin_66...,primary_aim_t3,T3 MRI study included in the ACRIN-6698 primar...,int64,"[{'option_code': '0', 'option_description': ' ..."


`table_name` gives us the location of the BigQuery table that contains the column described in the `column_metadata` row.

Given the information available in the per-collection clinical data, we can proceed with selecting a subset of patients that meet the criteria of your interest. As an example, the following query will select all of the distinct combination of patient ID and tumor grade, as defined by the `sbrgrade` column contents.

Note that in this case we use `%%bigquery` magic to execute the query - this is done just to demonstrate an alternative ways to work with SQL. You can switch between this approach and BigQuery Python API based on your preference! 

When you use `%%bigquery` magic, the result of the query will be saved into a pandas dataframe you specify as the last argument to the call (`acrin_6698_tumors`, in the example below). If you skip that argument, the result will be displayed in the cell, but you cannot reuse it later.

In [ ]:
%%bigquery --project=$my_ProjectID acrin_6698_tumors

SELECT 
  DISTINCT(dicom_patient_id), sbrgrade
FROM 
  bigquery-public-data.idc_v11_clinical.acrin_6698_clinical


In [ ]:
acrin_6698_tumors

,dicom_patient_id,sbrgrade
0,453236,II (Intermediate)
1,229047,None
2,384705,II (Intermediate)
3,415631,III (High)
4,793283,None
...,...,...
380,765671,II (Intermediate)
381,962153,III (High)
382,711476,II (Intermediate)
383,361701,II (Intermediate)


We can next use `dicom_patient_id` to link clinical data with the imaging studies available for the given patient. The query becomes a bit more complex, since we need to join data across two tables.

In [ ]:
%%bigquery --project=$my_ProjectID acrin_6698_viwable

SELECT
  ANY_VALUE(PatientID) AS PatientID,
  STRING_AGG(DISTINCT(acrin_6698_clinical.sbrgrade)) as tumor_grade,
  STRING_AGG(DISTINCT(Modality)) AS modalities,
  COUNT(DISTINCT(SeriesInstanceUID)) AS num_series,
  ANY_VALUE(CONCAT("https://viewer.imaging.datacommons.cancer.gov/viewer/", StudyInstanceUID)) AS viewer_url
FROM
  bigquery-public-data.idc_v11.dicom_all AS dicom_all
JOIN
  bigquery-public-data.idc_v11_clinical.acrin_6698_clinical AS acrin_6698_clinical
ON
  # we are prepending the value of dicom_patient_id to work around the known issue that should 
  # be resolved in the next release of IDC, see https://github.com/ImagingDataCommons/ETL/issues/43
  dicom_all.PatientID = CONCAT("ACRIN-6698-",acrin_6698_clinical.dicom_patient_id)
GROUP BY
  StudyInstanceUID
ORDER BY
  PatientID

In [ ]:
acrin_6698_viwable

,PatientID,tumor_grade,modalities,num_series,viewer_url
0,ACRIN-6698-102212,III (High),"MR,SEG",19,https://viewer.imaging.datacommons.cancer.gov/...
1,ACRIN-6698-102212,III (High),"SEG,MR",19,https://viewer.imaging.datacommons.cancer.gov/...
2,ACRIN-6698-102212,III (High),"MR,SEG",15,https://viewer.imaging.datacommons.cancer.gov/...
3,ACRIN-6698-102212,III (High),"SEG,MR",19,https://viewer.imaging.datacommons.cancer.gov/...
4,ACRIN-6698-103939,III (High),"SEG,MR",18,https://viewer.imaging.datacommons.cancer.gov/...
...,...,...,...,...,...
1118,ACRIN-6698-995480,III (High),"MR,SEG",18,https://viewer.imaging.datacommons.cancer.gov/...
1119,ACRIN-6698-995480,III (High),"SEG,MR",14,https://viewer.imaging.datacommons.cancer.gov/...
1120,ACRIN-6698-995480,III (High),"SEG,MR",18,https://viewer.imaging.datacommons.cancer.gov/...
1121,ACRIN-6698-995480,III (High),"MR,SEG",18,https://viewer.imaging.datacommons.cancer.gov/...


Remember that Google Cloud [BigQuery console](https://console.cloud.google.com/bigquery) is very handy in interactive exploration of tables and debugging of the queries! Until you become familiar with BigQuery and SQL, it is not recommended that you debug queries from the notebook cells. 

As an exercise, try to modify the query above to include count of distinct values of `SOPInstanceUID` in the query result (that number will correspond to the number of files you would be downloading for a given study).

## Discovery mode

Sometime you may want to find whether specific clinical attribute is available for the imaging data you can find in IDC. 

We can start by looking at the distinct values of `column_label` (which in the general case will be either more descriptive, or identical to `column`).


In [ ]:
import pandas as pd

pd.DataFrame({"column_label":column_metadata_df["column_label"].unique()})

,column_label
0,T0 (baseline) MRI study included in collection
1,T1 (early-Tx) MRI study included in collection
2,T2 (inter-regimen) MRI study included in colle...
3,T3 (pre-surgery) MRI study included in collection
4,Patient included in the BMMR2 challenge traini...
...,...
4010,residual_tumor
4011,tss_code
4012,country
4013,other_malignancy_anatomic_site


Let's say we want to know which of the cases have information related to therapy. We can search column metadata for the presence of word "therapy" (since the terms in clinical data are not harmonized, we need to account for the variability in capitalization).

In [ ]:
column_metadata_df[column_metadata_df["column_label"].str.contains("[tT]herapy", na=False)][["column_label", "collection_id", "values"]]

,column_label,collection_id,values
1193,radiotherapy_total_treat_time,head_neck_radiomics_hn1,"[{'option_code': '31', 'option_description': N..."
1197,CCRT Chemotherapy Regimen,hnscc,"[{'option_code': 'Carbolatin weekly', 'option_..."
1246,OnStudy Therapy Surgery Procedure Title,rembrandt,"[{'option_code': ' --', 'option_description': ..."
1604,Induction Chemotherapy,hnscc,[]
1754,OnStudy Therapy Chemo Agent Name,rembrandt,[]
...,...,...,...
5353,Other.anticoagulation.therapy,covid_19_ny_sbu,"[{'option_code': 'apixaban', 'option_descripti..."
5355,Chemotherapy Regimen,hnscc,"[{'option_code': 'Carbolatin weekly', 'option_..."
5356,Chemotherapy Medication,hnscc_3dct_rt,"[{'option_code': 'Carboplatin', 'option_descri..."
5364,OnStudy Therapy Radiation Type,rembrandt,"[{'option_code': ' --', 'option_description': ..."


We observe that there are several collections that contain column named "Chemotherapy". Let's filter these values further, in order to identify subjects that underwent chemotherapy.

In [ ]:
column_metadata_df[column_metadata_df["column_label"].str.contains("[Cc]hemotherapy", na=False)][[ "collection_id", "table_name", "column", "column_label","values"]]

,collection_id,table_name,column,column_label,values
6867,hcc_tace_seg,bigquery-public-data.idc_v11_clinical.hcc_tace...,chemotherapy,chemotherapy used for TACE procedure,"[{'option_code': 'Cisplastin', 'option_descrip..."
6921,head_neck_radiomics_hn1,bigquery-public-data.idc_v11_clinical.head_nec...,chemotherapy_given,chemotherapy_given,"[{'option_code': 'concomitant', 'option_descri..."
6934,hnscc,bigquery-public-data.idc_v11_clinical.hnscc_cl...,ccrt_chemotherapy_regimen,CCRT Chemotherapy Regimen,"[{'option_code': 'Carbolatin weekly', 'option_..."
6953,hnscc,bigquery-public-data.idc_v11_clinical.hnscc_cl...,induction_chemotherapy,Induction Chemotherapy,[]
7016,hnscc,bigquery-public-data.idc_v11_clinical.hnscc_cl...,platinum_based_chemotherapy,Platinum-based chemotherapy,"[{'option_code': 'No', 'option_description': N..."
7042,hnscc,bigquery-public-data.idc_v11_clinical.hnscc_cl...,chemotherapy_regimen,Chemotherapy Regimen,"[{'option_code': 'Carbolatin weekly', 'option_..."
7071,hnscc_3dct_rt,bigquery-public-data.idc_v11_clinical.hnscc_3d...,chemotherapy_medication,Chemotherapy Medication,"[{'option_code': 'Carboplatin', 'option_descri..."
7240,nsclc_radiogenomics,bigquery-public-data.idc_v11_clinical.nsclc_ra...,chemotherapy,Chemotherapy,"[{'option_code': 'No', 'option_description': N..."
7315,opc_radiomics,bigquery-public-data.idc_v11_clinical.opc_radi...,chemotherapy_,Chemotherapy,"[{'option_code': 'Yes', 'option_description': ..."


From the table above we can observe that:

1. There are several collections that have clinical metadata related to chemotherapy regimen of the subject; 

2. There are no value sets assigned for the column `induction_chemotherapy` in the table `hnscc_clinical`. Value sets may be missing if the dictionary is not provided, or if there are a large number (>20) distinct values of the column in the table. In such cases one can check the distinct values of the column by querying the table directly.  

Looking at the value sets for the collections/columns that have those, we can observe that subjects that had any chemotherapy could be selected as follows for the respective collections (non-exhaustive list):
* `head_neck_radiomics_hn1`: subjects that have value other than `none` in table `head_neck_radiomics_hn1_clinical` column `chemotherapy_given`
* `hnscc`: subjects that have value other than `No` in table `hnscc_clinical` column `chemotherapy_regime`
* `hnscc_3dct_rt`: subjects that have value other than `None` in table `hnscc_3dct_rt_demographics` column `chemotherapy_medication`
* `nsclc_radiogenomics`: subjects that have value `Yes` in table `nsclc_radiogenomics_clinical` column `chemotherapy`
* `hcc_tace_seg` column `chemotherapy` in the `hcc_tace_seg_clinical` table

Let's focus on the clinical data related to chemotherapy for the collection `hcc_tace_seg`.

In [ ]:
chemotherapy_subset = column_metadata_df[column_metadata_df["column_label"].str.contains("[Cc]hemotherapy", na=False)][[ "collection_id", "table_name", "column", "column_label","values"]]

chemotherapy_subset = chemotherapy_subset[chemotherapy_subset["collection_id"] == "hcc_tace_seg"]

chemotherapy_subset

,collection_id,table_name,column,column_label,values
6867,hcc_tace_seg,bigquery-public-data.idc_v11_clinical.hcc_tace...,chemotherapy,chemotherapy used for TACE procedure,"[{'option_code': 'Cisplastin', 'option_descrip..."


Before we select subjects that meet the criteria defined above, let's confirm  the values encountered in table `hcc_tace_seg_clinical` column `chemotherapy` match the value set in the `column_metadata` table. Here are the values of the dictionary we observe from the `column_metadata` inventory.

In [ ]:
chemotherapy_subset["values"].tolist()

[[{'option_code': 'Cisplastin', 'option_description': None},
  {'option_code': 'Cisplatin, Mitomycin-C', 'option_description': None},
  {'option_code': 'Cisplatin, doxorubicin, Mitomycin-C',
   'option_description': None},
  {'option_code': 'doxorubicin LC beads', 'option_description': None},
  {'option_code': 'null', 'option_description': None}]]

The query below will select the distinct values encountered in the `chemotherapy` column of the `hcc_tace_seg_clinical` table, which we confirm matches those in the dictionary.

In [ ]:
selection_query = """
SELECT DISTINCT(chemotherapy)
FROM bigquery-public-data.idc_v11_clinical.hcc_tace_seg_clinical
"""

selection_result = bq_client.query(selection_query)
hcc_tace_seg_clinical_df = selection_result.result().to_dataframe()
hcc_tace_seg_clinical_df

,chemotherapy
0,"Cisplatin, doxorubicin, Mitomycin-C"
1,None
2,doxorubicin LC beads
3,Cisplastin
4,"Cisplatin, Mitomycin-C"


Next we can examine the data to see what therapy individual patients had.

In [ ]:
selection_query = """
SELECT DISTINCT(dicom_patient_id), chemotherapy
FROM bigquery-public-data.idc_v11_clinical.hcc_tace_seg_clinical
WHERE chemotherapy <> "null"
"""

selection_result = bq_client.query(selection_query)
hcc_tace_seg_clinical_df = selection_result.result().to_dataframe()
hcc_tace_seg_clinical_df

,dicom_patient_id,chemotherapy
0,HCC_001,"Cisplatin, doxorubicin, Mitomycin-C"
1,HCC_006,"Cisplatin, doxorubicin, Mitomycin-C"
2,HCC_010,"Cisplatin, doxorubicin, Mitomycin-C"
3,HCC_017,"Cisplatin, doxorubicin, Mitomycin-C"
4,HCC_026,"Cisplatin, doxorubicin, Mitomycin-C"
...,...,...
83,HCC_050,"Cisplatin, doxorubicin, Mitomycin-C"
84,HCC_065,"Cisplatin, doxorubicin, Mitomycin-C"
85,HCC_073,doxorubicin LC beads
86,HCC_085,doxorubicin LC beads


`dicom_patient_id` is the key to connect clinical data and imaging data. Let's see what imaging studies we have for patient `HCC_101` that was treated with 	doxorubicin LC beads. Along the way we can also generate viewer URLs to conveniently examine the images.

In [ ]:
%%bigquery --project=$my_ProjectID

SELECT 
  StudyInstanceUID, 
  STRING_AGG(DISTINCT(Modality)) AS modalities, 
  STRING_AGG(DISTINCT(collection_id)) AS collection_id, 
  STRING_AGG(DISTINCT(Access)) AS access,
  COUNT(DISTINCT(SeriesInstanceUID)) AS num_series,
  ANY_VALUE(CONCAT("https://viewer.imaging.datacommons.cancer.gov/viewer/", StudyInstanceUID)) as viewer_url
FROM bigquery-public-data.idc_v11.dicom_all
WHERE PatientID = "HCC_103"
GROUP BY StudyInstanceUID

,StudyInstanceUID,modalities,collection_id,access,num_series,viewer_url
0,1.3.6.1.4.1.14519.5.2.1.1706.8374.304819071818...,"CT,SEG",hcc_tace_seg,Public,3,https://viewer.imaging.datacommons.cancer.gov/...
1,1.3.6.1.4.1.14519.5.2.1.1706.8374.121752675166...,CT,hcc_tace_seg,Public,2,https://viewer.imaging.datacommons.cancer.gov/...


## Patient-focused exploration

Sometime you may want to know whether specific patient has any clinical data available. One way to do this is to locate the collection that patient belongs to, and then check whether any of the clinical data tables (if any) that are available for that collection have that patient identifier.

Alternatively, we can build a complete list of patients that have clinical data by performing a union on all of the `dicom_patient_id` columns across all of the clinical data tables, which is what we do in the next cell.

In [ ]:
all_clinical_tables = column_metadata_df["table_name"].unique()
query = "with patients_unionized as (SELECT dicom_patient_id FROM "+all_clinical_tables[0].replace("v11","current")
for clinical_table in all_clinical_tables[1:]:
  query = query+" UNION ALL SELECT dicom_patient_id FROM "+clinical_table.replace("v11","current")

selection_query = query+") select distinct(dicom_patient_id) from patients_unionized"

#print(selection_query)

selection_result = bq_client.query(selection_query)
patients_df = selection_result.result().to_dataframe()


['TCGA-RZ-AB0B' 'TCGA-WC-A87T' 'TCGA-WC-A885' ... 'C3L-00997' 'C3N-01754'
 'C3N-00871']


In [ ]:
patients = patients_df["dicom_patient_id"].unique().tolist()

print("\n".join(patients))

TCGA-RZ-AB0B
TCGA-WC-A87T
TCGA-WC-A885
TCGA-WC-A883
TCGA-VD-AA8S
TCGA-YZ-A980
TCGA-V4-A9EF
TCGA-4V-A9QN
TCGA-5K-AAAP
TCGA-X7-A8DC
TCGA-X7-A8M3
TCGA-XU-A92V
TCGA-BJ-A191
TCGA-DJ-A3UT
TCGA-DJ-A3UM
TCGA-DJ-A1QI
TCGA-DJ-A3UN
TCGA-E8-A242
TCGA-E8-A2EA
TCGA-EL-A3T6
TCGA-EL-A3CX
TCGA-EM-A22M
TCGA-ET-A39R
TCGA-FY-A3I5
TCGA-EM-A2CO
TCGA-ET-A2N5
TCGA-FY-A3RA
TCGA-DJ-A3UR
TCGA-DJ-A2PR
TCGA-E3-A3DY
TCGA-EL-A3CW
TCGA-EM-A22O
TCGA-EM-A2CU
TCGA-ET-A3BO
TCGA-FK-A3SB
TCGA-FY-A3R7
TCGA-DJ-A2Q5
TCGA-2G-AALX
TCGA-2G-AAGY
TCGA-2G-AAGC
TCGA-2G-AAEW
TCGA-2G-AAGW
TCGA-2G-AAGG
TCGA-2G-AAFZ
TCGA-N5-A4RU
TCGA-N9-A4Q1
TCGA-N9-A4Q8
TCGA-NA-A4QX
TCGA-FX-A48G
TCGA-KD-A5QS
TCGA-QQ-A5VB
TCGA-DX-A6YV
TCGA-K1-A6RU
TCGA-SI-A71O
TCGA-3R-A8YX
TCGA-WK-A8XZ
TCGA-SI-A71Q
TCGA-KF-A41W
TCGA-FX-A76Y
TCGA-3B-A9HJ
TCGA-IF-A4AK
TCGA-DX-A6BH
TCGA-DX-A6BA
TCGA-QQ-A8VF
TCGA-DX-A3LU
TCGA-DX-A3LW
TCGA-Z4-AAPG
TCGA-FL-A1YM
TCGA-AX-A063
TCGA-AX-A05T
TCGA-A5-A0G9
TCGA-AP-A0LD
TCGA-B5-A0K6
TCGA-B5-A0JV
TCGA-B5-A11X
TCGA-A5-A0GG
TCGA-B5-A11O

**CAUTION:** Note that due to a [known issue](https://github.com/ImagingDataCommons/ETL/issues/43) where `dicom_patient_id` was not populated correctly at least for one collection, the IDs reported above may not be correct. We are working to resolve that issue and ensure accuracy of the identifiers.

# Want to learn more?

* check out other notebooks: https://github.com/ImagingDataCommons/IDC-Examples/tree/master/notebooks
* join our community forum to ask any questions about IDC: https://discourse.canceridc.dev/
* ask your questions during live discussions with IDC developers at the IDC weekly office hours - join us on Google Meet at https://meet.google.com/xyt-vody-tvb every Tuesday 16:30 – 17:30 (New York) and Wednesday 10:30-11:30 (New York)
* browse IDC portal: https://imaging.datacommons.cancer.gov/explore/
* read IDC paper: http://dx.doi.org/10.1158/0008-5472.CAN-21-0950
* watch a recent presentation about IDC: https://youtu.be/P9ateg9ZUEs